In [1]:
# so we can use packages from parent directory
import sys
sys.path.append("..")

In [2]:
#import skorch
import torch
import torch.nn as nn

import numpy as np
from monroe_data import MonroeData, MonroeDataEntry, Color # last two for reading pkl file
import caption_featurizers
from color_featurizers import ColorFeaturizer, color_phi_fourier
from experiment import FeatureHandler#, evaluate_model
import evaluation
from models import CaptionEncoder, LiteralListener, PytorchModel
from models import ColorSelector, ColorOnlyBaseline

In [166]:
import importlib
importlib.reload(evaluation)
import evaluation

In [187]:
import models
importlib.reload(models)
import models

In [3]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [12]:
ll = LiteralListener(CaptionEncoder)

In [19]:
gs = GridSearchCV(ll, param_grid, refit=False, cv=3)

In [16]:
param_grid = {"lr":[0.001, 0.004, 0.01], 'color_hidden_size':[50, 100, 150]}

In [18]:
list(ParameterGrid(param_grid))

[{'color_hidden_size': 50, 'lr': 0.001},
 {'color_hidden_size': 50, 'lr': 0.004},
 {'color_hidden_size': 50, 'lr': 0.01},
 {'color_hidden_size': 100, 'lr': 0.001},
 {'color_hidden_size': 100, 'lr': 0.004},
 {'color_hidden_size': 100, 'lr': 0.01},
 {'color_hidden_size': 150, 'lr': 0.001},
 {'color_hidden_size': 150, 'lr': 0.004},
 {'color_hidden_size': 150, 'lr': 0.01}]

In [21]:
import sklearn.linear_model.logistic as lr



In [22]:
e = lr.LogisticRegression()

In [23]:
e.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [102]:
default_args_map = {}

In [174]:
glove_weights = caption_featurizers.get_pretrained_glove(feature_handler.caption_featurizer.caption_indexer.idx2word.items(), prefix=True)

In [4]:
# parameter maps:

def init_default_arg_map(feature_handler):
    global default_args_map
    default_args_map = {
        # wrappers
        'PytorchModel': {'num_epochs' : 5,
                       'optimizer' : torch.optim.Adam,
                       'lr' : 0.004,
                       'criterion': torch.nn.CrossEntropyLoss
                      },
        # modules
        'CaptionEncoder': { 'embed_dim' : 100,
                          'hidden_dim' : 100,
                          'vocab_size' : feature_handler.caption_featurizer.caption_indexer.size,
                          'color_dim' : 54
                        },
        'ColorGenerator' : { 'embed_dim' : 100,
                            'hidden_dim' : 100,
                            'vocab_size' : feature_handler.caption_featurizer.caption_indexer.size,
                            'color_in_dim' : 54,
                            'color_hidden_dim' : 100,
                            'weight_matrix' : glove_weights
                            },
        'ColorSelector' : { 'color_dim' : 54 }
        
    }

def fill_default_args(model, params):
    model_default_params = {}
    # add args that should be there and filter out args that shouldn't
    for superclass in model.mro(): # mro = "method resolution order - it lists all the superclasses in order
        default_params = default_args_map.get(superclass.__name__, {})
        model_default_params = dict(default_params, **model_default_params)
        
    # filter out params with keys not in default
    params = {k: params[k] for k in params.keys() & model_default_params.keys()}
    params = dict(model_default_params, **params)
    
    return params
#     for klass in default_args_map.keys():
#         if isinstance(model, klass):
#             default_params = default_arg_map[klass]
#             params = dict(default_params, **params) # extend dict
#     return params
            

In [5]:
def retrain_model(wrapper, model, train_X, train_y, parameters, verbose=True):
    wrapper_args = fill_default_args(wrapper, parameters)
    model_args = fill_default_args(model, parameters)
    if verbose: print(wrapper_args, model_args)
    # create new model
    new_model = wrapper(model, **wrapper_args)
    new_model.init_model(**model_args)
    
    # train model
    new_model.fit(train_X, train_y)
    
    return new_model

In [6]:
def hyperparameter_search(search_parameters, wrapper, model, train_dev_data, score_model_f):
    # ~initialization code~
    train_X, train_y, assess_X, assess_y = train_dev_data # for now, for debugging
    
    best_model_score = -float("inf")
    best_model = None
    best_model_params = None
    tested_params = []

    print("Searching parameter space")
    for parameters in search_parameters:
        # retrain candidate model
        candidate_model = retrain_model(wrapper, model, train_X, train_y, parameters)
        # evaluate candidate model
        print("Evaluating Candidate Model:")
        candidate_model_score = score_model_f(candidate_model, assess_X, assess_y)
        print("Parameters: {}\tScore: {}".format(parameters, candidate_model_score))
        # store best model, score, parameters
        if candidate_model_score > best_model_score:
            best_model = candidate_model
            best_model_params = parameters
            best_model_score = candidate_model_score
        candidate_model.save_model("hyperparam_tuning_imagin/{}.params".format(parameters))    
        tested_params.append((parameters, candidate_model_score))
    
    return best_model, best_model_params, tested_params
        
    
        

In [7]:
def assess_model(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]

    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return reg_results[0] # just return the rho value


In [133]:
import copy

In [139]:
dev_data_synth.data.shape

(25850, 36)

In [8]:
# test on literal listener:
train_data = MonroeData("../data/csv/train_corpus_monroe.csv", "../data/entries/train_entries_monroe.pkl")
dev_data_synth = MonroeData("../data/csv/dev_corpus_synth_10fold.csv", "../data/entries/dev_corpus_synth_10fold.pkl")


In [9]:
caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer) # Use with parameter files that end in `endings_tkn` - using endings tokenizer to separate endings like "ish" and "er"
color_phi = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)
feature_handler = FeatureHandler(train_data, dev_data_synth, caption_phi, color_phi)

output_to_score_target = lambda model_output, target: np.exp(model_output[target]) # get the model's predicted probablity at each target index and use that as the score



In [103]:
init_default_arg_map(feature_handler)

In [88]:
test_params = {'color_dim':54}
fill_default_args(LiteralListener, test_params)

{'criterion': torch.nn.modules.loss.CrossEntropyLoss,
 'foobar': 4,
 'lr': 0.004,
 'num_epochs': 5,
 'optimizer': torch.optim.adam.Adam}

In [161]:
from functools import partial
literal_listener_assess_model = partial(assess_model, output_to_score=output_to_score_target)

In [96]:
literal_listener_assess_model

functools.partial(<function assess_model at 0x132c95c80>, output_to_score=<function <lambda> at 0x14a6a7620>)

In [ ]:
fill_default_args(LiteralListener, test_params)

In [91]:
param_grid = {"lr":[0.001, 0.004], 'color_hidden_size':[50, 100], 'num_epochs':[1]}
pg = list(ParameterGrid(param_grid))

In [92]:
pg

[{'color_hidden_size': 50, 'lr': 0.001, 'num_epochs': 1},
 {'color_hidden_size': 50, 'lr': 0.004, 'num_epochs': 1},
 {'color_hidden_size': 100, 'lr': 0.001, 'num_epochs': 1},
 {'color_hidden_size': 100, 'lr': 0.004, 'num_epochs': 1}]

In [107]:
print("Initializing data")
train_X = feature_handler.train_features()
train_y = feature_handler.train_targets()
assess_X = feature_handler.test_features()
assess_y = feature_handler.test_targets()

Initializing data


In [164]:
x = hyperparameter_search(pg, LiteralListener, CaptionEncoder, [train_X, train_y, assess_X, assess_y], literal_listener_assess_model)

Searching parameter space
{'num_epochs': 1, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_dim': 100, 'hidden_dim': 100, 'vocab_size': 974, 'color_dim': 54}
---EPOCH 0---
AFTER EPOCH 99 - AVERAGE VALIDATION LOSS: 1.679647059449926
Evaluating Candidate Model:
Parameters: {'color_hidden_size': 50, 'lr': 0.001, 'num_epochs': 1}	Score: -0.23228980693006493
{'num_epochs': 1, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.004, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_dim': 100, 'hidden_dim': 100, 'vocab_size': 974, 'color_dim': 54}
---EPOCH 0---
AFTER EPOCH 99 - AVERAGE VALIDATION LOSS: 1.8203433629265056
Evaluating Candidate Model:
Parameters: {'color_hidden_size': 50, 'lr': 0.004, 'num_epochs': 1}	Score: -0.11769071730060483
{'num_epochs': 1, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_d

In [165]:
x

(<models.LiteralListener at 0x14ef410b8>,
 {'color_hidden_size': 100, 'lr': 0.004, 'num_epochs': 1},
 [({'color_hidden_size': 50, 'lr': 0.001, 'num_epochs': 1},
   -0.23228980693006493),
  ({'color_hidden_size': 50, 'lr': 0.004, 'num_epochs': 1},
   -0.11769071730060483),
  ({'color_hidden_size': 100, 'lr': 0.001, 'num_epochs': 1},
   -0.03434776707752851),
  ({'color_hidden_size': 100, 'lr': 0.004, 'num_epochs': 1},
   0.15623398834939042)])

In [188]:
# Let's hyperparameter search for the best imaginative listener
from models import ImaginativeListener, ColorGenerator

In [223]:
def assess_model_abs(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]

    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return abs(reg_results[0]) # just return the rho value



In [224]:
caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer) # Use with parameter files that end in `endings_tkn` - using endings tokenizer to separate endings like "ish" and "er"
color_phi = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)
def target_color_target(data_entry):
    return np.array(data_entry.colors[0].rgb_norm)

feature_handler = FeatureHandler(train_data, dev_data_synth, caption_phi, color_phi, target_fn=target_color_target, randomized_colors=False)


output_to_score_de = lambda output, target: evaluation.delta_e_dist(output, target) # get the delta e distance between the target and the generated
imaginative_listener_assess_model = partial(assess_model_abs, output_to_score=output_to_score_de)

In [220]:
train_X = feature_handler.train_features()
train_y = feature_handler.train_targets()
assess_X = feature_handler.test_features()
assess_y = feature_handler.test_targets()

In [185]:
init_default_arg_map(feature_handler)

In [227]:
param_grid = {"lr":[0.0005, 0.001, 0.004, 0.01], 'color_hidden_dim':[50, 100, 150, 200], 'hidden_dim':[50, 100, 150, 200], # these do change
              'weight_matrix':[None, glove_weights], 'num_epochs':[2], 'criterion':[torch.nn.MSELoss]} # these don't change
pg = list(ParameterGrid(param_grid))
x = hyperparameter_search(pg, ImaginativeListener, ColorGenerator,
                          [train_X, train_y, assess_X, assess_y], imaginative_listener_assess_model)

Searching parameter space
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0005, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>} {'embed_dim': 100, 'hidden_dim': 50, 'vocab_size': 974, 'color_in_dim': 54, 'color_hidden_dim': 50, 'weight_matrix': None}
---EPOCH 0---
0m 0s (0:0 0.00%) 0.0001
0m 5s (0:1000 7.90%) 0.0573
0m 11s (0:2000 15.79%) 0.0528
0m 17s (0:3000 23.69%) 0.0556
0m 23s (0:4000 31.58%) 0.0532
0m 29s (0:5000 39.48%) 0.0559
0m 36s (0:6000 47.37%) 0.0488
0m 43s (0:7000 55.27%) 0.0455
0m 50s (0:8000 63.17%) 0.0496
0m 57s (0:9000 71.06%) 0.0441
1m 4s (0:10000 78.96%) 0.0466
1m 11s (0:11000 86.85%) 0.0451
1m 19s (0:12000 94.75%) 0.0450
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.047106276446565364
---EPOCH 1---
1m 26s (1:0 0.00%) 0.0000
1m 33s (1:1000 7.90%) 0.0447
1m 41s (1:2000 15.79%) 0.0458
1m 48s (1:3000 23.69%) 0.0497
1m 56s (1:4000 31.58%) 0.0488
2m 3s (1:5000 39.48%) 0.0503
2m 11s (1:6000 47.37%) 0.0454
2m 18s (1:7000 55.27%) 0.0429
2m 26s 

0m 3s (0:1000 7.90%) 0.0576
0m 9s (0:2000 15.79%) 0.0546
0m 14s (0:3000 23.69%) 0.0568
0m 20s (0:4000 31.58%) 0.0560
0m 27s (0:5000 39.48%) 0.0581
0m 34s (0:6000 47.37%) 0.0512
0m 41s (0:7000 55.27%) 0.0469
0m 48s (0:8000 63.17%) 0.0527
0m 55s (0:9000 71.06%) 0.0466
1m 3s (0:10000 78.96%) 0.0490
1m 10s (0:11000 86.85%) 0.0477
1m 18s (0:12000 94.75%) 0.0474
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.05018471184419468
---EPOCH 1---
1m 25s (1:0 0.00%) 0.0000
1m 33s (1:1000 7.90%) 0.0486
1m 41s (1:2000 15.79%) 0.0505
1m 49s (1:3000 23.69%) 0.0532
1m 57s (1:4000 31.58%) 0.0523
2m 4s (1:5000 39.48%) 0.0554
2m 12s (1:6000 47.37%) 0.0492
2m 20s (1:7000 55.27%) 0.0463
2m 28s (1:8000 63.17%) 0.0511
2m 35s (1:9000 71.06%) 0.0456
2m 43s (1:10000 78.96%) 0.0479
2m 51s (1:11000 86.85%) 0.0468
2m 58s (1:12000 94.75%) 0.0470
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04927963252866175
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 50, 'criterion': <class

1m 18s (0:10000 78.96%) 0.0460
1m 27s (0:11000 86.85%) 0.0452
1m 36s (0:12000 94.75%) 0.0452
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.047238089330804846
---EPOCH 1---
1m 45s (1:0 0.00%) 0.0000
1m 53s (1:1000 7.90%) 0.0452
2m 3s (1:2000 15.79%) 0.0455
2m 12s (1:3000 23.69%) 0.0493
2m 22s (1:4000 31.58%) 0.0486
2m 31s (1:5000 39.48%) 0.0509
2m 40s (1:6000 47.37%) 0.0456
2m 49s (1:7000 55.27%) 0.0429
2m 58s (1:8000 63.17%) 0.0463
3m 7s (1:9000 71.06%) 0.0419
3m 16s (1:10000 78.96%) 0.0433
3m 25s (1:11000 86.85%) 0.0433
3m 34s (1:12000 94.75%) 0.0428
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.046822633686475455
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 50, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 100, 'lr': 0.0005, 'num_epochs': 2, 'weight_matrix': None}	Score: 0.8834894812319889
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0005, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>}

2m 4s (1:1000 7.90%) 0.0488
2m 14s (1:2000 15.79%) 0.0502
2m 25s (1:3000 23.69%) 0.0549
2m 35s (1:4000 31.58%) 0.0536
2m 45s (1:5000 39.48%) 0.0582
2m 56s (1:6000 47.37%) 0.0515
3m 6s (1:7000 55.27%) 0.0470
3m 16s (1:8000 63.17%) 0.0524
3m 26s (1:9000 71.06%) 0.0468
3m 37s (1:10000 78.96%) 0.0487
3m 47s (1:11000 86.85%) 0.0471
3m 57s (1:12000 94.75%) 0.0476
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.05000525619182736
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 50, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 100, 'lr': 0.004, 'num_epochs': 2, 'weight_matrix': None}	Score: 0.8488908682169926
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.004, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>} {'embed_dim': 100, 'hidden_dim': 100, 'vocab_size': 974, 'color_in_dim': 54, 'color_hidden_dim': 50, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.

4m 31s (1:10000 78.96%) 0.0433
4m 43s (1:11000 86.85%) 0.0435
4m 56s (1:12000 94.75%) 0.0428
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.046196705841769777
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 50, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 150, 'lr': 0.0005, 'num_epochs': 2, 'weight_matrix': None}	Score: 0.885293023488378
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0005, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>} {'embed_dim': 100, 'hidden_dim': 150, 'vocab_size': 974, 'color_in_dim': 54, 'color_hidden_dim': 50, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
        ...,
        [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
        [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.

...

1m 23s (0:9000 71.06%) 0.0469
1m 34s (0:10000 78.96%) 0.0486
1m 45s (0:11000 86.85%) 0.0488
1m 56s (0:12000 94.75%) 0.0478
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.05163075420209983
---EPOCH 1---
2m 7s (1:0 0.00%) 0.0000
2m 18s (1:1000 7.90%) 0.0473
2m 30s (1:2000 15.79%) 0.0497
2m 41s (1:3000 23.69%) 0.0531
2m 53s (1:4000 31.58%) 0.0522
3m 5s (1:5000 39.48%) 0.0562
3m 17s (1:6000 47.37%) 0.0486
3m 28s (1:7000 55.27%) 0.0454
3m 40s (1:8000 63.17%) 0.0510
3m 52s (1:9000 71.06%) 0.0464
4m 4s (1:10000 78.96%) 0.0480
4m 16s (1:11000 86.85%) 0.0467
4m 28s (1:12000 94.75%) 0.0465
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04887509361227664
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 200, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 100, 'lr': 0.004, 'num_epochs': 2, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.270

2m 50s (1:1000 7.90%) 0.0446
3m 4s (1:2000 15.79%) 0.0454
3m 18s (1:3000 23.69%) 0.0492
3m 33s (1:4000 31.58%) 0.0482
3m 47s (1:5000 39.48%) 0.0515
4m 1s (1:6000 47.37%) 0.0449
4m 15s (1:7000 55.27%) 0.0423
4m 29s (1:8000 63.17%) 0.0462
4m 43s (1:9000 71.06%) 0.0420
4m 57s (1:10000 78.96%) 0.0434
5m 11s (1:11000 86.85%) 0.0437
5m 25s (1:12000 94.75%) 0.0426
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04567546447847659
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 200, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 150, 'lr': 0.0005, 'num_epochs': 2, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
        ...,
        [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
        [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883]

5m 48s (1:9000 71.06%) 0.0453
6m 5s (1:10000 78.96%) 0.0466
6m 22s (1:11000 86.85%) 0.0472
6m 40s (1:12000 94.75%) 0.0464
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04842542989854701
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 200, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 150, 'lr': 0.004, 'num_epochs': 2, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
        ...,
        [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
        [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
        [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
       dtype=torch.float64)}	Score: 0.842161309729904
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.01, 'criterion': <class 'torch.nn.module

Got here to composite score
Parameters: {'color_hidden_dim': 200, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>, 'hidden_dim': 200, 'lr': 0.0005, 'num_epochs': 2, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
        ...,
        [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
        [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
        [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
       dtype=torch.float64)}	Score: 0.8848891485689291
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>} {'embed_dim': 100, 'hidden_dim': 200, 'vocab_size': 974, 'color_in_dim': 54, 'color_hidden_dim': 200, 'weight_matrix': None}
---EPOCH 0---
0m 0s (0:0 0.00%) 0.0001
0m 10s (0:1000 7.90%) 0.0562


0m 11s (0:1000 7.90%) 0.0821
0m 29s (0:2000 15.79%) 0.0811
0m 47s (0:3000 23.69%) 0.0822
1m 7s (0:4000 31.58%) 0.0808
1m 43s (0:5000 39.48%) 0.2527
2m 33s (0:6000 47.37%) 0.3031
3m 19s (0:7000 55.27%) 0.3021
4m 7s (0:8000 63.17%) 0.3063
4m 55s (0:9000 71.06%) 0.3051
5m 42s (0:10000 78.96%) 0.3132
6m 26s (0:11000 86.85%) 0.3100
7m 11s (0:12000 94.75%) 0.3130
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.3065863100171943
---EPOCH 1---
7m 46s (1:0 0.00%) 0.0004
8m 30s (1:1000 7.90%) 0.3010
9m 20s (1:2000 15.79%) 0.3014
10m 13s (1:3000 23.69%) 0.3192
11m 6s (1:4000 31.58%) 0.3056
11m 56s (1:5000 39.48%) 0.3073
12m 47s (1:6000 47.37%) 0.3031
13m 35s (1:7000 55.27%) 0.3021
14m 24s (1:8000 63.17%) 0.3063
15m 12s (1:9000 71.06%) 0.3051
15m 59s (1:10000 78.96%) 0.3132
16m 44s (1:11000 86.85%) 0.3100
17m 29s (1:12000 94.75%) 0.3130
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.3065863100171943
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 200, 'criterio

In [236]:
imaginative_listener_assess_model(x[0], assess_X, asses_y)

Got here to composite score


0.8899286506479596

In [229]:
x[0]

In [230]:
x[1]

{'color_hidden_dim': 50,
 'criterion': torch.nn.modules.loss.MSELoss,
 'hidden_dim': 50,
 'lr': 0.001,
 'num_epochs': 2,
 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
         [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
         ...,
         [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
         [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
         [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
        dtype=torch.float64)}

In [232]:
x[2]

[({'color_hidden_dim': 50,
   'criterion': torch.nn.modules.loss.MSELoss,
   'hidden_dim': 50,
   'lr': 0.0005,
   'num_epochs': 2,
   'weight_matrix': None},
  0.8758884824550119),
 ({'color_hidden_dim': 50,
   'criterion': torch.nn.modules.loss.MSELoss,
   'hidden_dim': 50,
   'lr': 0.0005,
   'num_epochs': 2,
   'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
           [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
           [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
           ...,
           [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
           [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
           [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
          dtype=torch.float64)},
  0.8838690154613397),
 ({'color_hidden_dim': 50,
   'criterion': torch.nn.modules.loss.MSELoss,
   'hidden_dim': 50,
   'lr': 0.001,
   'num_epochs': 2,
   'weight_matrix': None},


In [221]:
param_grid = {"lr":[0.0005, 0.001], 'color_hidden_dim':[100], 'hidden_dim':[50], # these do change
              'weight_matrix':[glove_weights], 'num_epochs':[2], 'criterion':[torch.nn.MSELoss]} # these don't change
pg = list(ParameterGrid(param_grid))
x = hyperparameter_search(pg, ImaginativeListener, ColorGenerator,
                          [train_X, train_y, assess_X, assess_y], imaginative_listener_assess_model)

Searching parameter space
{'num_epochs': 2, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0005, 'criterion': <class 'torch.nn.modules.loss.MSELoss'>} {'embed_dim': 100, 'hidden_dim': 50, 'vocab_size': 974, 'color_in_dim': 54, 'color_hidden_dim': 100, 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
        ...,
        [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
        [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
        [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
       dtype=torch.float64)}
---EPOCH 0---
AFTER EPOCH 99 - AVERAGE VALIDATION LOSS: 0.07361920205876231
---EPOCH 1---
AFTER EPOCH 99 - AVERAGE VALIDATION LOSS: 0.07361920211464167
Evaluating Candidate Model:
Got here to composite score
Parameters: {'color_hidden_dim': 100, 'criterion': <c

In [222]:
x

(<models.ImaginativeListener at 0x15dd74748>,
 {'color_hidden_dim': 100,
  'criterion': torch.nn.modules.loss.MSELoss,
  'hidden_dim': 50,
  'lr': 0.0005,
  'num_epochs': 2,
  'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
          [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
          [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
          ...,
          [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
          [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
          [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
         dtype=torch.float64)},
 [({'color_hidden_dim': 100,
    'criterion': torch.nn.modules.loss.MSELoss,
    'hidden_dim': 50,
    'lr': 0.0005,
    'num_epochs': 2,
    'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
            [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
            [-0.1925,  0.4

In [228]:
import pickle
with open("imag_listener_hp_tune.pkl", "wb") as file:
    pickle.dump(x, file)

In [235]:
x[0].save_model("../model/imaginative_listener_hyper_param_tune.params")

# Let's now do the training thing, for 5 epochs! like 10 times

In [258]:
def sample_trained_model(parameters, wrapper, model, train_dev_data, score_model_f, num_samples=10):
    # ~initialization code~
    train_X, train_y, assess_X, assess_y = train_dev_data # for now, for debugging
    
    all_model_scores = []

    #print("Searching parameter space")
    for i in range(num_samples):
        # retrain candidate model
        candidate_model = retrain_model(wrapper, model, train_X, train_y, parameters, verbose=False)
        # evaluate candidate model
        print("Evaluating Candidate Model:")
        candidate_model_score = score_model_f(candidate_model, assess_X, assess_y)
        print("\t\tCorrelation\tMean\tStd")
        print("Iteration #{} Score: {}".format(i, candidate_model_score))
        
        candidate_model.save_model("../sampling/sample_{}.params".format(i))
        all_model_scores.append(candidate_model_score)
    
    return all_model_scores
        

In [237]:
def assess_model_mean_score(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]

    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return reg_results[0], np.mean(assess_model_scores), np.std(assess_model_scores)

In [ ]:
# use for literal listener (and maybe pragmatic listener too to get the correlation and accuracy)
def assess_model_accuracy(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]
    accuracy=[np.argmax(assess_model_outputs[i]) == assess_targets[i] for i in range(len(assess_model_outputs))]
    
    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return reg_results[0], np.mean(accuracy)

In [238]:
assess_model_mean_score(x[0], assess_X, assess_y, output_to_score_de)

Got here to composite score


(-0.8899286506479596, 24.388113753927573, 15.111289720886719)

In [264]:
best_params = x[1]
best_params["num_epochs"] = 5

In [246]:
imaginative_listener_assess_model_mean_score = partial(assess_model_mean_score, output_to_score=output_to_score_de)

In [266]:
all_imaginative_model_scores_5epoch = sample_trained_model(best_params, ImaginativeListener, ColorGenerator, [train_X, train_y, assess_X, assess_y], 
                     imaginative_listener_assess_model_mean_score, num_samples=10)


---EPOCH 0---
0m 0s (0:0 0.00%) 0.0001
0m 3s (0:1000 7.90%) 0.0600
0m 9s (0:2000 15.79%) 0.0531
0m 15s (0:3000 23.69%) 0.0541
0m 21s (0:4000 31.58%) 0.0523
0m 28s (0:5000 39.48%) 0.0554
0m 35s (0:6000 47.37%) 0.0477
0m 41s (0:7000 55.27%) 0.0449
0m 48s (0:8000 63.17%) 0.0491
0m 56s (0:9000 71.06%) 0.0438
1m 3s (0:10000 78.96%) 0.0457
1m 11s (0:11000 86.85%) 0.0450
1m 18s (0:12000 94.75%) 0.0442
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.0458541305232017
---EPOCH 1---
1m 26s (1:0 0.00%) 0.0000
1m 34s (1:1000 7.90%) 0.0445
1m 42s (1:2000 15.79%) 0.0452
1m 50s (1:3000 23.69%) 0.0486
1m 58s (1:4000 31.58%) 0.0479
2m 6s (1:5000 39.48%) 0.0505
2m 14s (1:6000 47.37%) 0.0449
2m 21s (1:7000 55.27%) 0.0424
2m 29s (1:8000 63.17%) 0.0454
2m 37s (1:9000 71.06%) 0.0422
2m 45s (1:10000 78.96%) 0.0432
2m 52s (1:11000 86.85%) 0.0435
3m 0s (1:12000 94.75%) 0.0426
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.046035199545983535
---EPOCH 2---
3m 8s (2:0 0.00%) 0.0000
3m 17s (2:1000 7.90%) 0.0429
3m 25s

0m 3s (0:1000 7.90%) 0.0599
0m 8s (0:2000 15.79%) 0.0533
0m 14s (0:3000 23.69%) 0.0544
0m 20s (0:4000 31.58%) 0.0524
0m 26s (0:5000 39.48%) 0.0559
0m 33s (0:6000 47.37%) 0.0478
0m 39s (0:7000 55.27%) 0.0451
0m 46s (0:8000 63.17%) 0.0493
0m 53s (0:9000 71.06%) 0.0436
1m 0s (0:10000 78.96%) 0.0453
1m 8s (0:11000 86.85%) 0.0451
1m 15s (0:12000 94.75%) 0.0442
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04599558908507849
---EPOCH 1---
1m 22s (1:0 0.00%) 0.0000
1m 29s (1:1000 7.90%) 0.0446
1m 38s (1:2000 15.79%) 0.0454
1m 46s (1:3000 23.69%) 0.0485
1m 54s (1:4000 31.58%) 0.0478
2m 2s (1:5000 39.48%) 0.0506
2m 9s (1:6000 47.37%) 0.0448
2m 16s (1:7000 55.27%) 0.0424
2m 24s (1:8000 63.17%) 0.0458
2m 31s (1:9000 71.06%) 0.0421
2m 39s (1:10000 78.96%) 0.0432
2m 46s (1:11000 86.85%) 0.0434
2m 53s (1:12000 94.75%) 0.0426
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.046023133295665805
---EPOCH 2---
3m 0s (2:0 0.00%) 0.0000
3m 8s (2:1000 7.90%) 0.0432
3m 15s (2:2000 15.79%) 0.0439
3m 23s (2:3000 2

0m 4s (0:1000 7.90%) 0.0596
0m 9s (0:2000 15.79%) 0.0530
0m 15s (0:3000 23.69%) 0.0544
0m 22s (0:4000 31.58%) 0.0521
0m 29s (0:5000 39.48%) 0.0558
0m 36s (0:6000 47.37%) 0.0478
0m 43s (0:7000 55.27%) 0.0448
0m 50s (0:8000 63.17%) 0.0490
0m 58s (0:9000 71.06%) 0.0437
1m 6s (0:10000 78.96%) 0.0455
1m 14s (0:11000 86.85%) 0.0451
1m 22s (0:12000 94.75%) 0.0442
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04609688331660194
---EPOCH 1---
1m 29s (1:0 0.00%) 0.0000
1m 37s (1:1000 7.90%) 0.0445
1m 45s (1:2000 15.79%) 0.0456
1m 53s (1:3000 23.69%) 0.0486
2m 1s (1:4000 31.58%) 0.0477
2m 9s (1:5000 39.48%) 0.0503
2m 17s (1:6000 47.37%) 0.0450
2m 25s (1:7000 55.27%) 0.0425
2m 33s (1:8000 63.17%) 0.0460
2m 40s (1:9000 71.06%) 0.0420
2m 48s (1:10000 78.96%) 0.0431
2m 56s (1:11000 86.85%) 0.0436
3m 4s (1:12000 94.75%) 0.0427
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.046000921015938126
---EPOCH 2---
3m 11s (2:0 0.00%) 0.0000
3m 20s (2:1000 7.90%) 0.0430
3m 28s (2:2000 15.79%) 0.0438
3m 36s (2:3000

0m 3s (0:1000 7.90%) 0.0594
0m 8s (0:2000 15.79%) 0.0532
0m 14s (0:3000 23.69%) 0.0543
0m 20s (0:4000 31.58%) 0.0522
0m 26s (0:5000 39.48%) 0.0556
0m 33s (0:6000 47.37%) 0.0478
0m 40s (0:7000 55.27%) 0.0444
0m 47s (0:8000 63.17%) 0.0494
0m 54s (0:9000 71.06%) 0.0434
1m 1s (0:10000 78.96%) 0.0456
1m 8s (0:11000 86.85%) 0.0448
1m 15s (0:12000 94.75%) 0.0441
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04619090907729696
---EPOCH 1---
1m 22s (1:0 0.00%) 0.0000
1m 29s (1:1000 7.90%) 0.0446
1m 37s (1:2000 15.79%) 0.0451
1m 45s (1:3000 23.69%) 0.0483
1m 52s (1:4000 31.58%) 0.0481
2m 0s (1:5000 39.48%) 0.0504
2m 7s (1:6000 47.37%) 0.0447
2m 15s (1:7000 55.27%) 0.0423
2m 22s (1:8000 63.17%) 0.0458
2m 30s (1:9000 71.06%) 0.0419
2m 37s (1:10000 78.96%) 0.0433
2m 44s (1:11000 86.85%) 0.0434
2m 52s (1:12000 94.75%) 0.0425
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.04580231395270675
---EPOCH 2---
2m 59s (2:0 0.00%) 0.0000
3m 6s (2:1000 7.90%) 0.0428
3m 14s (2:2000 15.79%) 0.0435
3m 22s (2:3000 2

In [267]:
all_imaginative_model_scores_5epoch

[(-0.8799177945947197, 24.225927799225367, 15.655397948223134),
 (-0.8869492153771232, 24.39706478328978, 15.507864792411981),
 (-0.8864349467822915, 24.16552102650053, 15.212897636506652),
 (-0.8849845382987327, 24.16229340966229, 15.429136297608911),
 (-0.888521525766309, 24.407595341399535, 15.319008963582462),
 (-0.8844849124233423, 24.339076564165975, 15.328759200193112),
 (-0.883466453189706, 24.34486000150015, 15.529266043787281),
 (-0.8863860909863949, 24.206922408749524, 15.49631615036218),
 (-0.8888535737157944, 24.193732947809874, 15.155575227648956),
 (-0.8813707874786111, 24.277307830774852, 15.463481141794134)]

In [263]:
#Parameters:
x[1]

{'color_hidden_dim': 50,
 'criterion': torch.nn.modules.loss.MSELoss,
 'hidden_dim': 50,
 'lr': 0.001,
 'num_epochs': 5,
 'weight_matrix': tensor([[-0.3111,  0.2883, -0.8058,  ..., -0.6888,  0.2855,  0.0024],
         [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.1925,  0.4819,  0.2420,  ..., -0.2026, -0.5078, -0.2903],
         ...,
         [-0.7525,  0.3095,  0.5609,  ..., -0.4011,  0.1240, -1.0467],
         [ 0.9683,  0.6163, -0.2813,  ...,  0.6563, -0.3829,  0.3883],
         [ 0.4379,  0.2097,  0.5134,  ..., -0.1190,  0.4062,  0.0903]],
        dtype=torch.float64)}

In [260]:
all_imaginative_model_scores = [(-0.8849432510288928, 24.309822169950884, 15.501392668130217),
 (-0.8840310193224501, 24.464459852528808, 15.335669354724137),
 (-0.8815876022537171, 24.460924854074598, 15.194064536632743),
 (-0.8827400800318522, 24.43559710182572, 15.304951314184677),
 (-0.8825997434197185, 24.596099200643636, 15.133469867131984),
 (-0.8832116627396694, 24.452777848555563, 15.408657721005932),
 (-0.8837957153292879, 24.3218522700263, 15.17006876530486),
 (-0.8810855717297843, 24.3237784045082, 15.310200595076472),
 (-0.883321931388866, 24.42361385599818, 15.559462627096627),
 (-0.8828887045800959, 24.742669023451597, 15.127002571761667)]

## Now let's do it with the baseline!!!

In [ ]:
baseline_listener_assess_model_mean_score = partial(assess_model_mean_score, output_to_score=output_to_score_de)

In [ ]:
all_baseline_model_scores_5epoch = sample_trained_model({}, ColorOnlyBaseline, ColorSelector, [train_X, train_y, assess_X, assess_y], 
                     imaginative_listener_assess_model_mean_score, num_samples=10)

